<IMG SRC="https://github.com/jacquesroy/byte-size-data-science/raw/master/images/Banner.png" ALT="BSDS Banner" WIDTH=1195 HEIGHT=200>

<table align="left">
    <tr><td>
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a></td><td>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.</td>
    </tr>
    <tr><td>Jacques Roy, Byte Size Data Science</td><td> </td></tr>
    </table>

# Spatial operations
We can process geo-spatial information using geopandas.

Geo-spatial data is complex and a lot can be done to manipulate it. 

### 049-Spatial Operations
Execute the next cell if you want to see the `Byte Size Data Science` youtube channel video

In [ ]:
from IPython.display import IFrame

IFrame(src="https://www.youtube.com/embed/llns01gm0AM?rel=0&amp;controls=0&amp;showinfo=0", width=560, height=315)


In [ ]:
!pip install geopandas 2>&1 >pipgeopandas.txt
import geopandas as gp

In [ ]:
!pip install folium 2>&1 >foliumpip.out

import folium

In [ ]:
import pandas as pd
import requests, zipfile, io

## Get the list of states to find the code for Oregon

In [ ]:
# States
url = 'http://www2.census.gov/geo/docs/reference/state.txt'
states_pd = pd.read_csv(url, sep='|')
print ("Number of states: " + str(states_pd['STATE'].count()))
states_pd[states_pd['STUSAB'] == 'OR']

## Get the list of counties for Oregon

In [ ]:
# Counties information
zip_file='https://www2.census.gov/geo/tiger/TIGER2019/COUNTY/tl_2019_us_county.zip'
r = requests.get(zip_file)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [ ]:
gdf_counties = gp.read_file('tl_2019_us_county.dbf')
print("Number of records: " + str(gdf_counties['STATEFP'].count()))
gdf_counties[gdf_counties['STATEFP'] == '41']

# Display the Multnomah county on a map

In [ ]:
Multnomah_geom = gdf_counties.iloc[319].geometry
centroid = Multnomah_geom.centroid
latlong = [centroid.y, centroid.x]

portland_map = folium.Map(location=[latlong[0], latlong[1]], zoom_start=10, width="80%", height="80%")

folium.GeoJson(
        Multnomah_geom,
        name='Multnomah county',
        tooltip='Multnomah county'
    ).add_to(portland_map)
folium.LayerControl().add_to(portland_map)
portland_map

## Get the road information for Multtnomah county

In [ ]:
# Multnomah County, COUNTYFP: 051, STATE: 41
# Roads information
!rm tl_*
zip_file='https://www2.census.gov/geo/tiger/TIGER2019/ROADS/tl_2019_41051_roads.zip'
r = requests.get(zip_file)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [ ]:
gdf_roads = gp.read_file('tl_2019_41051_roads.dbf')
print("Number of records: " + str(gdf_roads['LINEARID'].count()))
# Remove the records that don't have a name
gdf_roads = gdf_roads[gdf_roads['FULLNAME'].notna()]
print("Number of records with road names: " + str(gdf_roads['LINEARID'].count()))
gdf_roads.head()

## W Burnside Street

In [ ]:
# Find W Burnside Street
# This is where you can find Powell City of Books
gdf_roads[gdf_roads['FULLNAME'].str.contains("^[Ww] [Bb]urnside St")]

In [ ]:
# shapely.geometry.linestring.LineString
# See: https://shapely.readthedocs.io/en/stable/manual.html
geom = gdf_roads.iloc[1137].geometry
type(geom)

## Extract some attributes

In [ ]:
print("Type: " + geom.geom_type)
print("Length: " + str(geom.length))
print("Area: " + str(geom.area))
print("Bounds: " + str(geom.bounds))
print("Representative point: " + str(geom.representative_point()))

In [ ]:
# Bounding box
print('Bounding box: ')
print(geom.envelope)
print("Envelope area: " + str(geom.envelope.area))

In [ ]:
# Minimum rotated rectangle
print('Minimum rotated rectangle: ')
print(geom.minimum_rotated_rectangle)
print('Minimum rotated rectangle area: ' + str(geom.minimum_rotated_rectangle.area))

In [ ]:
# latlong = [45.5230969, -122.6813250] # Powell City of books
latlong = [geom.representative_point().y, geom.representative_point().x]

portland_map = folium.Map(location=[latlong[0], latlong[1]], zoom_start=15, width="95%", height="95%")

folium.GeoJson(
        geom.envelope,
        name='Burnside street envelope',
        tooltip='Burnside street envelope'
    ).add_to(portland_map)
folium.GeoJson(
        geom.minimum_rotated_rectangle,
        name='Burnside street minimum rectangle',
        tooltip='Burnside street minimum rectangl'
    ).add_to(portland_map)

folium.LayerControl().add_to(portland_map)
portland_map

## Find intersections

In [ ]:
xxx = gdf_roads[gdf_roads['geometry'].intersects(geom)]
print("Number of records: " + str(xxx['LINEARID'].count()))
xxx.head()

## Intersects vs. crosses

In [ ]:
xxx = gdf_roads[gdf_roads['geometry'].crosses(geom)]
print("Number of records: " + str(xxx['LINEARID'].count()))
xxx.head()